## parse dataset

In [1]:
from datasets import load_dataset, load_metric # Import dataset import function for hugging face
dataset = load_dataset("surrey-nlp/PLOD-CW") # import the coursework dataset from

from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification, pipeline
from transformers import BatchEncoding
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

import evaluate
seqeval = evaluate.load("seqeval")
metric = load_metric("seqeval")

import numpy as np
import os

C:\Users\olive\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
C:\Users\olive\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
C:\Users\olive\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torc

In [2]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [3]:
v = dataset["train"][286:287]["tokens"][0]
input_tokens = tokenizer(v, is_split_into_words=True)
input_ids = input_tokens["input_ids"]
id_tokens = []
for token in input_ids:
    id_tokens.append(tokenizer.convert_ids_to_tokens(token))
print(v)
print(id_tokens)
print(input_ids)
print(len(input_ids))

['(', 'EGF', ',', 'epidermal', 'growth', 'factor', ';', 'TGF', ',', 'transforming', 'growth', 'factor', ';', 'BTC', ',', 'betacellulin', ';', 'HB', '-', 'EGF', ',', 'heparin', '-', 'binding', 'epidermal', 'growth', 'factor', '(', 'EGF)-like', 'growth', 'factor', ';', 'EREG', ',', 'epiregulin', ';', 'NRG1', ',', 'neuregulin-1', ';', 'NRG2', ',', 'neuregulin-2', ';', 'NRG3', ',', 'neuregulin-3', ';', 'NRG4', ',', 'neuregulin-4', ';', 'PLCγ', ',', 'phospholipase', 'C', 'type', 'gamma', ';', 'CAMK2B', ',', 'calcium', '/', 'calmodulin', 'dependent', 'protein', 'kinase', ';', 'PRKCB', ',', 'Protein', 'kinase', 'C', '-', 'beta', ';', 'STAT5', ',', 'Signal', 'transducer', 'and', 'activator', 'of', 'transcription', '5', ';', 'src', ',', 'Rous', 'sarcoma', 'virus', 'gene', ';', 'CRK', ',', 'C', 'T10', 'regulator', 'of', 'a', 'tyrosine', 'kinase', ';', 'NCL', ',', 'NCK', 'Adaptor', 'Protein', '2', ';', 'PTK2', ',', 'PTK2', 'protein', 'tyrosine', 'kinase', '2', ';', 'ABL2', ',', 'V', '-', 'Abl', '

In [4]:
train_dict = dataset["train"]
train_tokens = train_dict["tokens"]
train_pos_tags = train_dict["pos_tags"]
train_ner_tags = train_dict["ner_tags"]

validation_dict = dataset["validation"]
validation_tokens = validation_dict["tokens"]
validation_pos_tags = validation_dict["pos_tags"]
validation_ner_tags = validation_dict["ner_tags"]

test_dict = dataset["test"]
test_tokens = test_dict["tokens"]
test_pos_tags = test_dict["pos_tags"]
test_ner_tags = test_dict["ner_tags"]

def data_to_lower(data:list[list[str]]) -> list[list[str]]:
    return [[token.lower() for token in tokens] for tokens in data]

train_tokens = data_to_lower(train_tokens)
validation_tokens = data_to_lower(validation_tokens)
test_tokens = data_to_lower(test_tokens)

class DataItem:
    def __init__(self, tokens, pos, ner, idx=0):
        self.idx=idx
        self.tokens:list[str] = tokens
        self.pos:list[str] = pos
        self.ner:list = ner
        self.tokenised_inputs:BatchEncoding = tokenizer(self.tokens, is_split_into_words=True) # also contains attention mask!

    def get_as_tuple(self) -> tuple:
        return (self.tokens, self.pos, self.ner)
    
    def get_as_tuple_list(self) -> list[tuple]:
        tuple_list = []
        for idx in range(len(self.tokens)-1):
            tuple_list.append((self.tokens[idx], self.pos[idx], self.ner[idx]))
        return tuple_list
    
    def ner_label2idx(self, label2idx_dict):
        if not isinstance(self.ner[0], str):
            print("WARNING - NER not listed as labels! NER Type: ",type(self.ner[0]),", Exiting...")
            return
        for idx, ner in enumerate(self.ner):
            ner[idx] = label2idx_dict[ner]
    
    def ner_idx2label(self, idx2label_dict):
        if not isinstance(self.ner[0], int):
            print("WARNING - NER not listed as indecies! Exiting...")
            return
        for idx, ner in enumerate(self.ner):
            ner[idx] = idx2label_dict[ner]

class DataCollection:
    def __init__(self, data_collection:list[DataItem], max_token_length=512):
        self.max_token_length = max_token_length
        self.data_collection:list[DataItem] = data_collection
        self.unique_tags = self.get_unique_tags()
        self.item_embeddings:dict = self.create_item_embeddings(self.unique_tags)
        self.reverse_embeddings:dict = {v:k for k,v in self.item_embeddings.items()}

    def get_token_list(self) -> list[list[str]]:
        return [data_item.tokens for data_item in self.data_collection]

    def get_pos_list(self) -> list[list[str]]:
        return [data_item.pos for data_item in self.data_collection]

    def get_ner_list(self) -> list[list[str]]:
        return [data_item.ner for data_item in self.data_collection]
    
    def get_ner_idx_list(self) -> list[list[str]]:
        ner_idx_list_collection = []
        for data_item in self.data_collection:
            ner_idx_list = []
            for ner_tag in data_item.ner:
                ner_idx_list.append(self.item_embeddings[ner_tag])
            ner_idx_list_collection.append(ner_idx_list)
        return ner_idx_list_collection

    def get_unique_tags(self) -> list[str]:
        unique_list = []
        ner_tags_list:list = self.get_ner_list()
        for ner_list in ner_tags_list:
            for ner in ner_list:
                if ner not in unique_list:
                    unique_list.append(ner)
        return unique_list
    
    def create_item_embeddings(self, tags:list[str]) -> dict:
        return {label:idx for idx, label in enumerate(tags)}
    
    def get_invalid_token_lengths(self) -> list[int]:
        invalid_lengths = []
        for idx, data_item in enumerate(self.data_collection):
            if len(data_item.tokenised_inputs["input_ids"]) >= self.max_token_length:
                invalid_lengths.append(idx)
                print("Data item idx ",idx," has tokens longer than ",self.max_token_length)
        return invalid_lengths

    def remove_invalid_token_length_items(self) -> None:
        invalid_lengths = self.get_invalid_token_lengths()
        for index in sorted(invalid_lengths, reverse=True):
            del self.data_collection[index]

train_data:list[DataItem] = []
for idx in range(len(train_tokens)):
    train_data.append(DataItem(train_tokens[idx], train_pos_tags[idx], train_ner_tags[idx], idx))
train_collection:DataCollection = DataCollection(train_data)

validation_data:list[DataItem] = []
for idx in range(len(validation_tokens)):
    validation_data.append(DataItem(validation_tokens[idx], validation_pos_tags[idx], validation_ner_tags[idx], idx))
validation_collection:DataCollection = DataCollection(validation_data)

test_data:list[DataItem] = []
for idx in range(len(test_tokens)):
    test_data.append(DataItem(test_tokens[idx], test_pos_tags[idx], test_ner_tags[idx], idx))
test_collection:DataCollection = DataCollection(test_data)

In [5]:
train_collection.remove_invalid_token_length_items()
validation_collection.remove_invalid_token_length_items()
test_collection.remove_invalid_token_length_items()

Data item idx  286  has tokens longer than  512


In [6]:
# Following https://huggingface.co/docs/transformers/main/en/tasks/token_classification

# More on https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/token_classification.ipynb#scrollTo=IjyhFKjlEP_B

## Tokenization Example

Adds extra start and end tags (CLS and SEP), as well potentially splits one word into 2. Thus have to realign indecies.

We also have to assign -100 to CLS and SEP so they are ignored by PyTorch loss function (CrossEntropyLoss)

Only label first token of a word, add -100 for subtokens of the same word

In [7]:
example = dataset["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)
print(tokenized_input)

['[CLS]', 'for', 'this', 'purpose', 'the', 'gothenburg', 'young', 'persons', 'empowerment', 'scale', '(', 'g', '##ype', '##s', ')', 'was', 'developed', '.', '[SEP]']
{'input_ids': [101, 2005, 2023, 3800, 1996, 22836, 2402, 5381, 23011, 4094, 1006, 1043, 18863, 2015, 1007, 2001, 2764, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [8]:
def tokenize_and_align_labels(data_collection:DataCollection):
    tokenized_inputs = tokenizer(data_collection.get_token_list(), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(data_collection.get_ner_idx_list()):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_train = tokenize_and_align_labels(train_collection)
tokenized_validation = tokenize_and_align_labels(validation_collection)
tokenized_test = tokenize_and_align_labels(test_collection)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [9]:
label_list = train_collection.unique_tags
labels = train_collection.unique_tags


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

id2label = train_collection.reverse_embeddings
label2id = train_collection.item_embeddings

In [10]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(labels), id2label=id2label, label2id=label2id)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
def turn_dict_to_list_of_dict(d):
    new_list = []

    for labels, inputs in zip(d["labels"], d["input_ids"]):
        entry = {"input_ids": inputs, "labels": labels}
        new_list.append(entry)

    return new_list

tokenised_train = turn_dict_to_list_of_dict(tokenized_train)
tokenised_val = turn_dict_to_list_of_dict(tokenized_validation)
tokenised_test = turn_dict_to_list_of_dict(tokenized_test)

In [10]:
model_output_dir = "distilbert_model"

training_args = TrainingArguments(
    output_dir=model_output_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3, # number of epochs to train
    weight_decay=0.01, # The weight decay to apply (if not zero) to all layers except all bias and LayerNorm weights
    evaluation_strategy="epoch",
    save_strategy="epoch", # can save by epoch, steps or not at all
    save_total_limit=1, # how many checkpoints to keep before overriding (set to 1, so latest checkpoint is only kept)!
    load_best_model_at_end=True,
    report_to=['none'], # REQUIRED because otherwise keeps asking to log into "wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenised_train,
    eval_dataset=tokenised_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

C:\Users\olive\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/201 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/10 [00:00<?, ?it/s]

Checkpoint destination directory distilbert_model\checkpoint-67 already exists and is non-empty.Saving will proceed but saved results may be invalid.


...

  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.6835944652557373, 'eval_precision': 0.8786975952330283, 'eval_recall': 0.8259651930386077, 'eval_f1': 0.8515157764487524, 'eval_accuracy': 0.8256, 'eval_runtime': 0.1873, 'eval_samples_per_second': 816.698, 'eval_steps_per_second': 53.379, 'epoch': 2.0}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.7148677110671997, 'eval_precision': 0.879948914431673, 'eval_recall': 0.8269653930786157, 'eval_f1': 0.8526348355161391, 'eval_accuracy': 0.826, 'eval_runtime': 0.1742, 'eval_samples_per_second': 878.244, 'eval_steps_per_second': 57.402, 'epoch': 3.0}
{'train_runtime': 12.4892, 'train_samples_per_second': 257.262, 'train_steps_per_second': 16.094, 'train_loss': 0.30154730194243623, 'epoch': 3.0}


TrainOutput(global_step=201, training_loss=0.30154730194243623, metrics={'train_runtime': 12.4892, 'train_samples_per_second': 257.262, 'train_steps_per_second': 16.094, 'train_loss': 0.30154730194243623, 'epoch': 3.0})

In [11]:
trainer.evaluate()

  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.5402113795280457,
 'eval_precision': 0.8706697459584296,
 'eval_recall': 0.8295659131826365,
 'eval_f1': 0.849620979307519,
 'eval_accuracy': 0.8404,
 'eval_runtime': 0.4307,
 'eval_samples_per_second': 355.262,
 'eval_steps_per_second': 23.22,
 'epoch': 3.0}

In [12]:
# # Prepare the test data for evaluation in the same format as the training data

# predictions, labels, _ = trainer.predict(tokenised_test)
# predictions = np.argmax(predictions, axis=2)

# # label_list = test_collection.get_ner_idx_list()
# label_list = train_collection.unique_tags

# # Remove the predictions for the [CLS] and [SEP] tokens 
# true_predictions = [
#     [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
#     for prediction, label in zip(predictions, labels)
# ]
# true_labels = [
#     [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
#     for prediction, label in zip(predictions, labels)
# ]

# # Compute multiple metrics on the test restuls
# results = metric.compute(predictions=true_predictions, references=true_labels)
# results
predictions, labels, _ = trainer.predict(tokenised_val) # tokenized validation used instead of validation dataset (as recommended for vectorised)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

  0%|          | 0/8 [00:00<?, ?it/s]

{'AC': {'precision': 0.6996466431095406,
  'recall': 0.752851711026616,
  'f1': 0.7252747252747251,
  'number': 263},
 'LF': {'precision': 0.4311377245508982,
  'recall': 0.47368421052631576,
  'f1': 0.45141065830721,
  'number': 152},
 'O': {'precision': 0.9566947565543071,
  'recall': 0.9591645153719784,
  'f1': 0.9579280440642213,
  'number': 4261},
 'overall_precision': 0.9227022448115205,
 'overall_recall': 0.9317792985457656,
 'overall_f1': 0.9272185571398169,
 'overall_accuracy': 0.9128}

In [27]:
predictions

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 2, 2, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 3, 0, ..., 0, 0, 0]], dtype=int64)

In [13]:
for data_item in validation_collection.data_collection:
    t = data_item.tokens
    print(len(t))
    print(t)
    tok = tokenizer(t, is_split_into_words=True)
    print(len(tok["input_ids"]))
    print(tok) # tokenizer adds the CLS and SEP! So should be safe to rip out first and last character
    toks = []
    for to in tok["input_ids"]:
        toks.append(tokenizer.convert_ids_to_tokens(to))
    print(len(toks))
    print(toks)
    break

51
['=', 'manual', 'ability', 'classification', 'system', ';', 'quest', '=', 'quest', '-', 'quality', 'of', 'upper', 'extremity', 'skills', 'test', ';', 'cont', '=', 'control', ';', 'm', '=', 'male', ',', 'f', '=', 'female', ',', 'v', '=', 'verbal', ',', 'nonv', '=', 'non', '-', 'verbal', ',', '|quad', '=', 'quadriplegia', ',', 'di', '=', 'diplegia', ',', 'hemi', '=', 'hemiplegia', '.']
67
{'input_ids': [101, 1027, 6410, 3754, 5579, 2291, 1025, 8795, 1027, 8795, 1011, 3737, 1997, 3356, 4654, 7913, 16383, 4813, 3231, 1025, 9530, 2102, 1027, 2491, 1025, 1049, 1027, 3287, 1010, 1042, 1027, 2931, 1010, 1058, 1027, 12064, 1010, 2512, 2615, 1027, 2512, 1011, 12064, 1010, 1064, 17718, 1027, 17718, 29443, 23115, 2401, 1010, 4487, 1027, 16510, 23115, 2401, 1010, 19610, 2072, 1027, 19610, 11514, 23115, 2401, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [14]:
id2label[-100] = ""
validation_labels =  [label for label in tokenized_validation["labels"][0]]
print(validation_labels)
validation_label_ids = [id2label[label] for label in tokenized_validation["labels"][0]]
print(validation_label_ids)
print(len(validation_label_ids))

[-100, 0, 1, 2, 2, 2, 0, 3, 0, 1, 2, 2, 2, 2, 2, -100, -100, 2, 2, 0, 0, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 0, -100, 0, 0, -100, -100, -100, 0, 0, 0, 0, -100, -100, 0, 0, -100, 0, 0, -100, -100, -100, 0, -100]
['', 'B-O', 'B-LF', 'I-LF', 'I-LF', 'I-LF', 'B-O', 'B-AC', 'B-O', 'B-LF', 'I-LF', 'I-LF', 'I-LF', 'I-LF', 'I-LF', '', '', 'I-LF', 'I-LF', 'B-O', 'B-O', '', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', '', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', 'B-O', '', 'B-O', 'B-O', '', '', '', 'B-O', 'B-O', 'B-O', 'B-O', '', '', 'B-O', 'B-O', '', 'B-O', 'B-O', '', '', '', 'B-O', '']
67


In [15]:
idx_num = 1
sentances = []
for data_item in validation_collection.data_collection:
    sentances.append({"sentance":" ".join(data_item.tokens),"token_len":len(data_item.tokens)})
print("sentance:", sentances[idx_num]["sentance"])
print("token length:", sentances[idx_num]["token_len"])

tokenized_input = [data.tokenised_inputs for data in validation_collection.data_collection]
print("original length:", len(tokenized_input[idx_num]["input_ids"]))
token_values = []
for token in tokenized_input[idx_num]["input_ids"]:
    token_value = tokenizer.convert_ids_to_tokens(token)
    if token_value != "[CLS]" and token_value != "[SEP]":
        token_values.append(token_value)
print(token_values)
print("after removing cls and sep length:", len(token_values))

checkpoint_list:list[str] = os.listdir(model_output_dir)
last_checkpoint:str = checkpoint_list[-1:][0]
last_checkpoint_path:str = os.path.join(model_output_dir, last_checkpoint)
classifier = pipeline("ner", model=last_checkpoint_path)
result:list[dict] = classifier(sentances[idx_num]["sentance"])
print("predicted tokens length:",len(result))
print([value["entity"] for value in result])


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


sentance: electro - oculography ( eog ) ( retiport32 , roland consult , wiesbaden , germany ) was performed in all patients according to the guidelines of the international society for clinical electrophysiology of vision ( iscev).[12 ] arden ratios below 1.8 were rated as pathologic .
token length: 46
original length: 71
['electro', '-', 'o', '##cu', '##log', '##raphy', '(', 'e', '##og', ')', '(', 're', '##tip', '##ort', '##32', ',', 'roland', 'consult', ',', 'wi', '##es', '##bad', '##en', ',', 'germany', ')', 'was', 'performed', 'in', 'all', 'patients', 'according', 'to', 'the', 'guidelines', 'of', 'the', 'international', 'society', 'for', 'clinical', 'electro', '##phy', '##sio', '##logy', 'of', 'vision', '(', 'is', '##ce', '##v', ')', '.', '[', '12', ']', 'arden', 'ratios', 'below', '1', '.', '8', 'were', 'rated', 'as', 'path', '##olo', '##gic', '.']
after removing cls and sep length: 69
predicted tokens length: 69
['I-LF', 'I-LF', 'I-LF', 'I-LF', 'I-LF', 'I-LF', 'B-O', 'B-AC', 'B-A

In [16]:
sentances = []
for data_item in validation_collection.data_collection:
    sentances.append({"sentance":" ".join(data_item.tokens),"token_len":len(data_item.tokens)})

measure_dict = []
id2label = validation_collection.reverse_embeddings
tokenized_validation
for idx, tokenised in enumerate(tokenised_val):
    # d = {"tags":[], "sentance":"","classifier_res":[]}
    tokenised_l = tokenised["labels"][1:-1]
    tags = []
    prev_id = list(validation_collection.item_embeddings.keys())[0]
    for label_id in tokenised_l:
        if label_id != -100:
            tags.append(id2label[label_id])
            prev_id = id2label[label_id]
        else:
            tags.append(prev_id)
    tokenised_in = tokenised["input_ids"][1:-1]
    tokens = []
    for token in tokenised_in:
        tokens.append(tokenizer.convert_ids_to_tokens(token))
    result:list[dict] = classifier(sentances[idx]["sentance"])
    classifier_entities = [value["entity"] for value in result]
    measure_dict.append({"tags":tags, "tokens":tokens, "sentance":sentances[idx]["sentance"], "predicted_tags": classifier_entities})


In [17]:
idx_val = 3
print(measure_dict[idx_val]["sentance"])
print(len(measure_dict[idx_val]["tags"]))
print(len(measure_dict[idx_val]["predicted_tags"]))

t -snares syn-1a and snap25 through their interactions with pm - bound voltage - gated calcium channels ( cav ) , l - type in β - cells and n - type in neurons , position the predocked sgs to the site of maximum ca2 + influx for efficient exocytosis [ 6–12 ] .
71
71


In [18]:
tags = validation_collection.item_embeddings.keys()

class TagMetrics:
    def __init__(self, tag:str, tag_dict:dict):
        self.tag = tag
        self.correct = tag_dict["correct"]
        self.incorrect = tag_dict["incorrect"]
        self.total = tag_dict["total"]

class MetricItem:
    def __init__(self, total:int, correct:int, incorrect:int, tags_dict:dict, idx=0):
        self.idx:int = idx
        self.total:int = total
        self.correct:int = correct
        self.incorrect:int = incorrect
        self.tag_metric:list[TagMetrics] = [TagMetrics(tag, tags_dict[tag]) for tag in tags_dict.keys()]

class MetricCollection:
    def __init__(self, metric_items:list[MetricItem]):
        self.data_collection:list[MetricItem] = metric_items
        self.total_correct:int = sum([item.correct for item in metric_items]) 
        self.total_incorrect:int = sum([item.incorrect for item in metric_items])
        self.total_label_measurement:dict = self.__items_to_label_measure__()

    def __items_to_label_measure__(self) -> dict:
        tag_measure = {}
        for item in metric_items:
            for tag_metric in item.tag_metric:
                if tag_metric.tag not in tag_measure.keys():
                    tag_measure[tag_metric.tag] = {"correct":tag_metric.correct, "incorrect":tag_metric.incorrect, "total":tag_metric.total}
                else:
                    tag_measure[tag_metric.tag]["correct"] += tag_metric.correct
                    tag_measure[tag_metric.tag]["incorrect"] += tag_metric.incorrect
                    tag_measure[tag_metric.tag]["total"] += tag_metric.total
        return tag_measure

print("VALIDATION COLLECTION SIZE: ", len(validation_collection.data_collection))
total_tokens = 0
for idx in range(len(measure_dict)):
    total_tokens += len(measure_dict[idx]["tokens"])
print("TOTAL TOKENS IN VALIDATION SET:", total_tokens)


metric_items:list[MetricItem] = []
for idx in range(len(measure_dict)):
    correct = 0
    incorrect = 0
    tags_dict = {tag:{"correct":0, "incorrect":0, "total":0} for tag in tags}
    current_total = len(measure_dict[idx]["tokens"])
    for d_idx in range(current_total):
        tag = measure_dict[idx]["tags"][d_idx]
        predicted_tag = measure_dict[idx]["predicted_tags"][d_idx]
        if tag == predicted_tag:
            correct += 1
            tags_dict[tag]["correct"] += 1
            tags_dict[tag]["total"] += 1
        else:
            incorrect += 1
            tags_dict[tag]["incorrect"] += 1
            tags_dict[tag]["total"] += 1
    metric_items.append(MetricItem(current_total, correct, incorrect, tags_dict, idx=idx))
    # break # for only 1 item

metric_collection:MetricCollection = MetricCollection(metric_items)
print("TOTAL VALUES:", metric_collection.total_label_measurement)
# print("IDX:1")
# print("TOTAL CORRECT:",correct,"/",current_total)
# print("TOTAL INCORRECT:",incorrect,"/",current_total)
# for k in tags_dict.keys():
#     print(k, ":", tags_dict[k])

VALIDATION COLLECTION SIZE:  126
TOTAL TOKENS IN VALIDATION SET: 6549
TOTAL VALUES: {'B-O': {'correct': 4988, 'incorrect': 220, 'total': 5208}, 'B-LF': {'correct': 27, 'incorrect': 262, 'total': 289}, 'I-LF': {'correct': 355, 'incorrect': 134, 'total': 489}, 'B-AC': {'correct': 313, 'incorrect': 250, 'total': 563}}


In [19]:
import pandas as pd
tags = validation_collection.get_unique_tags()
df = pd.DataFrame(0, columns=tags, index=tags)

for idx in range(len(measure_dict)):
    for d_idx in range(len(measure_dict[idx]["tokens"])):
        tag = measure_dict[idx]["tags"][d_idx]
        predicted_tag = measure_dict[idx]["predicted_tags"][d_idx]
        df.at[tag, predicted_tag] += 1 # first is row, second is column (meaning rows are true tags and columns are predicted tags)
print(df) # center diagonally is TP (and TN) and everything else is FP (and FN)!

def calc_precision(TP, FP):
    return TP / (TP + FP)

def calc_recall(TP, FN):
    return TP / (TP + FN)

def calc_f1(precision, recall):
    return (2 * precision * recall) / (precision + recall)

total_correct = 0
for tag in tags:
    total_correct += df.at[tag,tag]

total_incorrect = total_tokens - total_correct

TP = 56 + 410 + 324
TN = 4969
FP = 104 + 1 + 7 + 77 + 0 + 232
FN = 19 + 126 + 1 + 132 + 3 + 88
# precision = calc_precision(total_correct, total_incorrect)
# recall = calc_recall(total_correct, total_incorrect)
# f1_score = calc_f1(precision, recall)

precision = calc_precision(TP, FP)
recall = calc_recall(TP, FN)
f1_score = calc_f1(precision, recall)

print("Precision:", precision)
print("Recall:", recall)
print("F1_Score:", f1_score)


       B-O  B-LF  I-LF  B-AC
B-O   4988     7   123    90
B-LF   115    27   137    10
I-LF   133     0   355     1
B-AC   241     0     9   313
Precision: 0.652353426919901
Recall: 0.6816220880069025
F1_Score: 0.6666666666666666


In [20]:
# Dataframe the first item in validation set!

data = []
for idx in range(len(measure_dict)):
    for d_idx in range(len(measure_dict[idx]["tokens"])):
        data.append([measure_dict[idx]["tokens"][d_idx], measure_dict[idx]["tags"][d_idx], measure_dict[idx]["predicted_tags"][d_idx]])
    break
df = pd.DataFrame(data, columns=['Tokens', 'Tags', "Predicted Tags"])
df.to_csv('result.csv', index=False)  
df

,Tokens,Tags,Predicted Tags
0,=,B-O,B-O
1,manual,B-LF,B-LF
2,ability,I-LF,I-LF
3,classification,I-LF,I-LF
4,system,I-LF,I-LF
...,...,...,...
60,hem,B-O,B-O
61,##ip,B-O,B-O
62,##leg,B-O,B-O
63,##ia,B-O,B-O
